In [ ]:
# Calculating metrics using speed filter --> eliminates registers speed < 2km and speed > 80

In [1]:
# filtering registers speed < 2km and speed > 80
for day in range(1,32):
    traces = spark.read.parquet("s3://mobility-traces-sp/processed-data-avl-date/10-speed-calculation-based-bus-location/MO_1510"+str(day)+"/")    
    traces_new = traces.filter("speed >= 2").filter("speed < 80")
    traces_new.write.parquet("s3://mobility-traces-sp/processed-data-avl-date/using-vehicle-2-80/20-speed-calculation-filtered/MO_1510"+str(day)+"/")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1611024810140_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Metrics Active Buses

In [2]:
# installing required packages for this notebook session
sc.install_pypi_package("boto3")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Total vehicles per day

import pyspark.sql.functions as F
import boto3

csv_out = "day,number_of_vehicles\n"

for day in range(1,32):
    traces = spark.read.parquet("s3://mobility-traces-sp/processed-data-avl-date/using-vehicle-2-80/20-speed-calculation-filtered/MO_1510"+str(day)+"/")
    n_vehicles_day = traces.select(F.countDistinct("id_avl").alias("count")).collect()[0]["count"]
    
    csv_out += f"MO_1510{day},{n_vehicles_day}\n"


s3 = boto3.client('s3')

# writing results in S3
s3.put_object(Body=bytes(csv_out,"utf-8"), Bucket='mobility-traces-sp', Key='metrics-calculation/using-speed-2-80/actives-buses/active-buses-per-day.csv')    


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': 'D41CC9E21B30B075', 'HostId': 'eH5jk0FAXdW3ZAplwiuU6WfjragHeh6gFSfe6WrXpf/PCxWElzzKaqHsWTZVecVUWqmhmZJ49UA=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'eH5jk0FAXdW3ZAplwiuU6WfjragHeh6gFSfe6WrXpf/PCxWElzzKaqHsWTZVecVUWqmhmZJ49UA=', 'x-amz-request-id': 'D41CC9E21B30B075', 'date': 'Tue, 19 Jan 2021 03:35:05 GMT', 'etag': '"fa62f25d0d50a8fef589cbb4a5fc87b6"', 'content-length': '0', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'ETag': '"fa62f25d0d50a8fef589cbb4a5fc87b6"'}

In [4]:
import pyspark.sql.functions as F

# Number of vehicles per hour per day
for day in range(1,32):
    traces = spark.read.parquet("s3://mobility-traces-sp/processed-data-avl-date/using-vehicle-2-80/20-speed-calculation-filtered/MO_1510"+str(day)+"/")
    df = traces.groupby("hour_avl").agg(F.countDistinct("id_avl").alias("number_buses"))
    df.write.parquet(f"s3://mobility-traces-sp/metrics-calculation/using-speed-2-80/actives-buses/active-buses-per-hour/MO_1510{day}/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
import pyspark.sql.functions as F

# Number of vehicles per hour and region per day
for day in range(1,32):
    traces = spark.read.parquet("s3://mobility-traces-sp/processed-data-avl-date/using-vehicle-2-80/20-speed-calculation-filtered/MO_1510"+str(day)+"/")
    df = traces.groupby("hour_avl","region").agg(F.countDistinct("id_avl").alias("number_buses"))
    df.write.parquet(f"s3://mobility-traces-sp/metrics-calculation/using-speed-2-80/actives-buses/active-buses-per-hour-per-region/MO_1510{day}/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Speed metrics

In [8]:
import pyspark.sql.functions as F

# calculating speed per vehicle
for day in range(1,32):
    traces = spark.read.parquet("s3://mobility-traces-sp/processed-data-avl-date/using-vehicle-2-80/20-speed-calculation-filtered/MO_1510"+str(day)+"/")
    df_speed = traces.groupby("id_avl","line_id").agg(F.avg("speed").alias("avg_speed"),F.stddev("speed").alias("speed_stddev"))
    df_speed.write.parquet("s3://mobility-traces-sp/metrics-calculation/using-speed-2-80/speed-calculation/speed-per-vehicle/MO_1510"+str(day)+"/")
    
    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# installing required packages for this notebook session
sc.install_pypi_package("boto3")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Package already installed for current Spark context!
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/context.py", line 1110, in install_pypi_package
    raise ValueError("Package already installed for current Spark context!")
ValueError: Package already installed for current Spark context!



In [10]:
# avg speed per day

import pyspark.sql.functions as F

# statistics speed per vehicle
csv_out_vehicle = "day,n_vehicles,n_lines,n_vehicles_lines,avg_speed,avg_speed_min,avg_speed_max,avg_speed_stddev,avg_speed_quantile_6.25,avg_speed_quantile_12.5,avg_speed_quantile_25,avg_speed_quantile_50,avg_speed_quantile_75\n"

for day in range(1,32):
    # speed by vehicle per line
    speeds_per_vehicle = spark.read.parquet("s3://mobility-traces-sp/metrics-calculation/using-speed-2-80/speed-calculation/speed-per-vehicle/MO_1510"+str(day)+"/")
    
    sizes = speeds_per_vehicle.select(F.countDistinct("id_avl").alias("vehicles"),
                                      F.countDistinct("line_id").alias("lines"),
                                      F.countDistinct("id_avl","line_id").alias("vehicles_lines")
                                    ).collect()
    
    stats = speeds_per_vehicle.agg(F.mean('avg_speed').alias('mean'),
                       F.min('avg_speed').alias('min'),
                       F.max('avg_speed').alias('max'),
                       F.stddev('avg_speed').alias("stddev")).collect()
    
    quantiles = speeds_per_vehicle.approxQuantile("avg_speed", [0.0625,0.125,0.25,0.5,0.75], 0.0001)
    
    csv_out_vehicle += "{},{},{},{},{},{},{},{},{},{},{},{},{}\n"\
        .format("MO_1510"+str(day),
                sizes[0]["vehicles"],sizes[0]["lines"],sizes[0]["vehicles_lines"],
                stats[0]["mean"],stats[0]["min"],stats[0]["max"],stats[0]["stddev"],
                quantiles[0],quantiles[1],quantiles[2],quantiles[3],quantiles[4]
        )

import boto3   
s3 = boto3.client('s3')

s3.put_object(Body=bytes(csv_out_vehicle,"utf-8"), Bucket='mobility-traces-sp', Key='metrics-calculation/using-speed-2-80/speed-calculation/statistics/1-speed_per_vehicle_stats.csv')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': 'B02806C6248827C6', 'HostId': 'qmAyY4UyGfZEX05e9mMiB4kzCSlePbHIuo5POdgeqGtP5r55GMGRZCIJ3B9Yp5reuspwJR+1iCQ=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'qmAyY4UyGfZEX05e9mMiB4kzCSlePbHIuo5POdgeqGtP5r55GMGRZCIJ3B9Yp5reuspwJR+1iCQ=', 'x-amz-request-id': 'B02806C6248827C6', 'date': 'Tue, 19 Jan 2021 03:54:09 GMT', 'etag': '"c2c202a14c24c3e359580ec191ad7658"', 'content-length': '0', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'ETag': '"c2c202a14c24c3e359580ec191ad7658"'}

In [11]:
# Calculating speed by hour per day
# 1- Calculating the avg speed per bus per hour --> groupby id_avl,line_id,hour
# 2- Calculating the avg speed per hour per day --> groupby hour
import pyspark.sql.functions as F

# calculating speed per hour per day
for day in range(1,32):
    # Reading traces
    traces = spark.read.parquet("s3://mobility-traces-sp/processed-data-avl-date/using-vehicle-2-80/20-speed-calculation-filtered/MO_1510"+str(day)+"/")
    
    # Calculating speed by vehicle per hour
    df_speed_hour_per_vehicle = traces.groupby("id_avl","line_id","hour_avl").agg(F.avg("speed").alias("avg_speed"),F.stddev("speed").alias("speed_stddev"))
    df_speed_hour_per_vehicle.write.parquet("s3://mobility-traces-sp/metrics-calculation/using-speed-2-80/speed-calculation/speed-per-vehicle-per-hour/MO_1510"+str(day)+"/")
    
    # Calculating speed by hour per day
    df_speed_hour_day = df_speed_hour_per_vehicle.groupby("hour_avl").agg(F.avg("avg_speed").alias("avg_speed"),F.stddev("avg_speed").alias("speed_stddev"))
    df_speed_hour_day.write.parquet("s3://mobility-traces-sp/metrics-calculation/using-speed-2-80/speed-calculation/speed-per-hour-per-day/MO_1510"+str(day)+"/")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# Calculating speed by region per day
# 1- Calculating the avg speed per bus per region --> groupby id_avl,line_id,region
# 2- Calculating the avg speed per region per day --> groupby region
import pyspark.sql.functions as F

# calculating speed per region per day
for day in range(1,32):
    # Reading traces
    traces = spark.read.parquet("s3://mobility-traces-sp/processed-data-avl-date/using-vehicle-2-80/20-speed-calculation-filtered/MO_1510"+str(day)+"/")
    
    # Calculating speed by vehicle per region
    df_speed_region_per_vehicle = traces.groupby("id_avl","line_id","region").agg(F.avg("speed").alias("avg_speed"),F.stddev("speed").alias("speed_stddev"))
    df_speed_region_per_vehicle.write.parquet("s3://mobility-traces-sp/metrics-calculation/using-speed-2-80/speed-calculation/speed-per-vehicle-per-region/MO_1510"+str(day)+"/")
    
    # Calculating speed by region per day
    df_speed_region_day = df_speed_region_per_vehicle.groupby("region").agg(F.avg("avg_speed").alias("avg_speed"),F.stddev("avg_speed").alias("speed_stddev"))
    df_speed_region_day.write.parquet("s3://mobility-traces-sp/metrics-calculation/using-speed-2-80/speed-calculation/speed-per-region-per-day/MO_1510"+str(day)+"/")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-12:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 1598



# Connectivity metrics

In [ ]:
# data preparation

In [ ]:
# identifying graph_id

In [13]:
import pyspark.sql.functions as F

for day in range(1,32):
    traces = spark.read.parquet("s3://mobility-traces-sp/processed-data-avl-date/using-vehicle-2-80/20-speed-calculation-filtered/MO_1510"+str(day)+"/")
    df = traces.repartition(150)
    
    df_graph_id = df\
        .withColumn('minute_avl',F.minute(F.col("dt_avl")))\
        .withColumn('graph_id',F.concat(F.col("hour_avl"),F.lit("-"),F.col("minute_avl"),F.lit("-"),F.col("region")))\
        .drop("hour_diff","dt_avl","speed")

    df_graph_id.repartition("graph_id").write.parquet("s3://mobility-traces-sp/processed-data-avl-date/using-vehicle-2-80/21-traces-graph-1-minute-with-graph-id/MO_1510"+str(day)+"/")
    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# joining the dataset

import pyspark.sql.functions as f

days = [23,24,25,26,27,28,29,30,31]

for day in days:
    df = spark.read.parquet("s3://mobility-traces-sp/processed-data-avl-date/using-vehicle-2-80/21-traces-graph-1-minute-with-graph-id/MO_1510"+str(day)+"/")

    df2 = df

    df.alias('df1').join(df2.alias("df2"),on=["graph_id"],how="outer")\
        .select(
            f.col("df1.id_avl").alias("id_avl_1"),
            f.col("df1.line_id").alias("line_1"),
            f.col("df1.latitude").alias("latitude_1"),
            f.col("df1.longitude").alias("longitude_1"),
            f.col("df2.id_avl").alias("id_avl_2"),
            f.col("df2.line_id").alias("line_2"),
            f.col("df2.latitude").alias("latitude_2"),
            f.col("df2.longitude").alias("longitude_2"),
            f.col("df1.hour_avl").alias("hour_avl"),
            f.col("df1.minute_avl").alias("minute_avl"),
            f.col("df1.region").alias("region"),
            f.col("graph_id").alias("graph_id"),

    ).write.parquet("s3://mobility-traces-sp/processed-data-avl-date/using-vehicle-2-80/22-joined-graph-1-minute/MO_1510"+str(day)+"/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
sc.install_pypi_package("haversine")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/f4/52/a13286844780c7b1740edbbee8a8f0524e2a6d51c068b59dda39a6a119f5/haversine-2.3.0-py2.py3-none-any.whl

In [4]:
from haversine import haversine, Unit
import pyspark.sql.functions as F

def get_distance(lat1,lon1,lat2,lon2):
    coord_1 = (lat1,lon1)
    coord_2 = (lat2,lon2)

    distance = haversine(coord_1,coord_2,unit=Unit.METERS)
    
    return distance
    
get_distance_udf = F.udf(get_distance)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# eliminating duplicates
for day in range(1,32):
    joined = spark.read.parquet("s3://mobility-traces-sp/processed-data-avl-date/using-vehicle-2-80/22-joined-graph-1-minute/MO_1510"+str(day)+"/")
    joined.repartition("graph_id").filter("id_avl_1 != id_avl_2").write.parquet("s3://mobility-traces-sp/processed-data-avl-date/using-vehicle-2-80/23-no-duplicated/MO_1510"+str(day)+"/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
import pyspark.sql.functions as F

days_to_analyze = [1,5,4,12,17,20]
for day in days_to_analyze:
    no_repeated = spark.read.parquet("s3://mobility-traces-sp/processed-data-avl-date/using-vehicle-2-80/23-no-duplicated/MO_1510"+str(day)+"/")
    distance = no_repeated.withColumn("distance",
                    get_distance_udf(F.col("latitude_1"),F.col("longitude_1"),F.col("latitude_2"),F.col("longitude_2")))

    df_final = distance.repartition("graph_id")

    df_final.filter("distance <= 100")\
        .repartition("graph_id")\
        .write.parquet("s3://mobility-traces-sp/processed-data-avl-date/using-vehicle-2-80/24-no-duplicated-with-all-distances/MO_1510"+str(day)+"/")

In [2]:
import pyspark.sql.functions as F

days_to_analyze = [1,5,4,12,17,20]
for day in days_to_analyze:
    distances = spark.read.parquet("s3://mobility-traces-sp/processed-data-avl-date/using-vehicle-2-80/24-no-duplicated-with-all-distances/MO_1510"+str(day)+"/")
    df = distances.drop_duplicates(subset=["graph_id","id_avl_1","id_avl_2"])
    df.repartition("graph_id").write.parquet("s3://mobility-traces-sp/processed-data-avl-date/using-vehicle-2-80/25-no-repeated-contact-only-100-distances/MO_1510"+str(day)+"/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Node Degree per vehicle per graph

days_to_analyze = [1,5,4,12,17,20]
for day in days_to_analyze:
    
    df = spark.read.parquet("s3://mobility-traces-sp/processed-data-avl-date/using-vehicle-2-80/25-no-repeated-contact-only-100-distances/MO_1510"+str(day)+"/")
    df_counts = df.groupby("id_avl_1","graph_id").agg(F.countDistinct("id_avl_2").alias("number_connections"))

    df_counts.write.parquet("s3://mobility-traces-sp/metrics-calculation/using-speed-2-80/connectivity-metrics/connections-per-vehicle-per-graph/MO_1510"+str(day)+"/")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# Node Degree avg per minute
import pyspark.sql.functions as f


days_to_analyze = [1,5,4,12,17,20]
for day in days_to_analyze:
    df = spark.read.parquet("s3://mobility-traces-sp/metrics-calculation/using-speed-2-80/connectivity-metrics/connections-per-vehicle-per-graph/MO_1510"+str(day)+"/") 

    df = df.withColumn('splitted', F.split(df['graph_id'], '-'))\
        .withColumn('hour', F.col('splitted')[0])\
        .withColumn('minute', F.col('splitted')[1])\
        .withColumn('hour-minute', F.concat(F.col('hour'),F.lit(":"),F.col("minute")))\
        .withColumn('region', F.col('splitted')[2])\
        .drop("splitted")
    
    df = df.groupby("hour-minute").agg(f.avg("number_connections").alias("avg_degree"))\
            .withColumn('time', F.date_format('hour-minute','HH:mm'))\
            .drop("hour-minute")
    
    df.write.parquet("s3://mobility-traces-sp/metrics-calculation/using-speed-2-80/connectivity-metrics/connections-degree-per-minute/MO_1510"+str(day)+"/")
    
    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# Degree vehicle per day statistics
import pyspark.sql.functions as F

days_to_analyze = [1,5,4,12,17,20]
for day in days_to_analyze:
    print("Day",day)
    count_connections = spark.read.parquet(f"s3://mobility-traces-sp/metrics-calculation/using-speed-2-80/connectivity-metrics/connections-per-vehicle-per-graph/MO_1510{day}/")

    count_connections.agg(
        F.avg("number_connections").alias("avg"),
        F.stddev("number_connections").alias("stddev"),
        F.max("number_connections").alias("max"),
        F.min("number_connections").alias("min"),
    ).show()

    quantiles = count_connections.approxQuantile("number_connections", [0.25,0.5,0.75], 0.0001)
    print(quantiles)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Day 1
+-----------------+-----------------+---+---+
|              avg|           stddev|max|min|
+-----------------+-----------------+---+---+
|3.690357977881169|3.396344978033525| 52|  1|
+-----------------+-----------------+---+---+

[1.0, 3.0, 5.0]
Day 5
+------------------+------------------+---+---+
|               avg|            stddev|max|min|
+------------------+------------------+---+---+
|3.7534078358463727|3.4847551992377506| 59|  1|
+------------------+------------------+---+---+

[1.0, 3.0, 5.0]
Day 4
+------------------+------------------+---+---+
|               avg|            stddev|max|min|
+------------------+------------------+---+---+
|2.3342890031601637|1.8588067594456918| 27|  1|
+------------------+------------------+---+---+

[1.0, 2.0, 3.0]
Day 12
+-----------------+------------------+---+---+
|              avg|            stddev|max|min|
+-----------------+------------------+---+---+
|2.396904215117114|1.9351793086571762| 28|  1|
+-----------------+-------

In [14]:
# Connection_id_column

import pyspark.sql.functions as F

# Creating a connection_id for each connection with the buses.
# Connection_id = id_avl_1+id_avl_2 if id_avl_1 < id_avl_2
# Connection_id = id_avl_2+id_avl_1 if id_avl_2 < id_avl_1

days_to_analyze = [1,5,4,12,17,20]

for day in days_to_analyze:

    df = spark.read.parquet("s3://mobility-traces-sp/processed-data-avl-date/using-vehicle-2-80/25-no-repeated-contact-only-100-distances/MO_1510"+str(day)+"/")

    df = df.withColumn("connection_id",
        F.when(F.col("id_avl_1") < F.col("id_avl_2"), F.concat(F.col("id_avl_1"),F.lit("-"),F.col("id_avl_2")))
        .otherwise(F.concat(F.col("id_avl_2"),F.lit("-"),F.col("id_avl_1"))))

    df = df.repartition("graph_id")



    df.write.parquet("s3://mobility-traces-sp/processed-data-avl-date/using-vehicle-2-80/26-no-repeated-contact-only-100-distances-with-connection-id/MO_1510"+str(day)+"/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
# Saving count of repeated connections

import pyspark.sql.functions as F

days_to_analyze = [1,5,4,12,17,20]

for day in days_to_analyze:

    df = spark.read.parquet("s3://mobility-traces-sp/processed-data-avl-date/using-vehicle-2-80/26-no-repeated-contact-only-100-distances-with-connection-id/MO_1510"+str(day)+"/")

    df = df.drop_duplicates(subset=["connection_id","graph_id"])

    df_counts = df.groupby("connection_id").agg(F.count("connection_id").alias("count_per_day"))

    df_counts.write.parquet(f"s3://mobility-traces-sp/metrics-calculation/using-speed-2-80/connectivity-metrics/repeated-connection-per-day/MO_1510{day}")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:

# Number of repeated connections per day/duration of the connections

import pyspark.sql.functions as F

days_to_analyze = [1,5,4,12,17,20]

for day in days_to_analyze:
    print("Day",day)
    df = spark.read.parquet(f"s3://mobility-traces-sp/processed-data-avl-date/using-vehicle-2-80/26-no-repeated-contact-only-100-distances-with-connection-id/MO_1510{day}/")

    df = df.drop_duplicates(subset=["connection_id","graph_id"])

    df_counts = df.groupby("connection_id").agg(F.count("connection_id").alias("count_per_day"))

    df_counts.agg(
        F.avg("count_per_day").alias("avg"),
        F.stddev("count_per_day").alias("stddev"),
        F.max("count_per_day").alias("max"),
        F.min("count_per_day").alias("min"),
    ).show()

    quantiles = df_counts.approxQuantile("count_per_day", [0.25,0.5,0.75], 0.0001)
    print(quantiles)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Day 1
+-----------------+-----------------+---+---+
|              avg|           stddev|max|min|
+-----------------+-----------------+---+---+
|2.641490255137404|3.357811855685901|166|  1|
+-----------------+-----------------+---+---+

[1.0, 1.0, 3.0]
Day 5
+-----------------+-----------------+---+---+
|              avg|           stddev|max|min|
+-----------------+-----------------+---+---+
|2.668634439659038|3.405101187213052|170|  1|
+-----------------+-----------------+---+---+

[1.0, 2.0, 3.0]
Day 4
+-----------------+----------------+---+---+
|              avg|          stddev|max|min|
+-----------------+----------------+---+---+
|2.656030673917721|3.17151251960479|151|  1|
+-----------------+----------------+---+---+

[1.0, 2.0, 3.0]
Day 12
+-----------------+-----------------+---+---+
|              avg|           stddev|max|min|
+-----------------+-----------------+---+---+
|2.667264523334527|3.215293056650444|100|  1|
+-----------------+-----------------+---+---+

[1.0, 2.

In [18]:
# Total number of distinct connections per day
import pyspark.sql.functions as F

days_to_analyze = [1,5,4,12,17,20]

for day in days_to_analyze:
    print("Day",day)
    connections = spark.read.parquet(f"s3://mobility-traces-sp/processed-data-avl-date/using-vehicle-2-80/26-no-repeated-contact-only-100-distances-with-connection-id/MO_1510{day}/")

    connections.agg(F.countDistinct("connection_id").alias("number_distinct_conn")).show()
    connections.groupby("graph_id","connection_id").agg(F.countDistinct("connection_id").alias("count_novo"))\
        .agg(F.sum("count_novo").alias("total_per_day")).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Day 1
+--------------------+
|number_distinct_conn|
+--------------------+
|             4007958|
+--------------------+

+-------------+
|total_per_day|
+-------------+
|     10586982|
+-------------+

Day 5
+--------------------+
|number_distinct_conn|
+--------------------+
|             4047723|
+--------------------+

+-------------+
|total_per_day|
+-------------+
|     10801893|
+-------------+

Day 4
+--------------------+
|number_distinct_conn|
+--------------------+
|              878401|
+--------------------+

+-------------+
|total_per_day|
+-------------+
|      2333060|
+-------------+

Day 12
+--------------------+
|number_distinct_conn|
+--------------------+
|              938352|
+--------------------+

+-------------+
|total_per_day|
+-------------+
|      2502833|
+-------------+

Day 17
+--------------------+
|number_distinct_conn|
+--------------------+
|             1908130|
+--------------------+

+-------------+
|total_per_day|
+-------------+
|      5101826|
